Source : https://databank.worldbank.org/source/world-development-indicators
Prédiction de la population

In [ ]:
import pandas as pd
from IPython.display import display
# InteractiveShell.ast_node_interactivity = "all"


data = pd.read_csv("pop_1960_2023.csv")


data = data.drop(["Series Code"], axis=1)

# utiliser missMDA pour gérer les Nan


rename_dict = {}
# Renommer colonnes : 1960 [YR1960] => YR1960
for col in data.columns:
    if col[0].isdigit():
        rename_dict[col] = col[:4]
data = data.rename(columns=rename_dict)
        
# remettre données année en ligne et variable en colonne



data = pd.melt(data, id_vars=list(data.columns[0:3]), value_vars = list(data.columns[3:]))
data = data.rename(columns={"variable":"Year"})
print(data.columns)

# data_unique = data.drop_duplicates(subset=["Country Name", "Country Code", "Year", "Series Name"])


# Pivot pour que les valeurs de "Series Name" deviennent des colonnes
data_pivot = data.pivot_table(
    index=["Country Name", "Country Code", "Year"],
    columns="Series Name",
    values="value",
    aggfunc='first'  
)

# a vérifier la logique 

data_pivot = data_pivot.reset_index()
data = data_pivot
display(data_pivot[data_pivot["Country Name"]=="Afghanistan"].head(200))


data.to_csv("data_final.csv", index=False)
len(data["Country Name"].unique())

# bien penser à séparer les agrégats des données de base (pays)

Index(['Country Name', 'Country Code', 'Series Name', 'Year', 'value'], dtype='object')


Series Name,Country Name,Country Code,Year,Access to electricity (% of population),Adjusted net national income (current US$),"Birth rate, crude (per 1,000 people)","Death rate, crude (per 1,000 people)",Electric power consumption (kWh per capita),Exports of goods and services (% of GDP),GDP per capita (current US$),"Life expectancy at birth, total (years)",Population growth (annual %),"Population, total","Suicide mortality rate (per 100,000 population)"
0,Afghanistan,AFG,1960,..,..,50.34,31.921,..,..,..,32.535,..,9035043,..
1,Afghanistan,AFG,1961,..,..,50.443,31.349,..,..,..,33.068,1.9622391202036,9214083,..
2,Afghanistan,AFG,1962,..,..,50.57,30.845,..,..,..,33.547,2.0445228359303,9404406,..
3,Afghanistan,AFG,1963,..,..,50.703,30.359,..,..,..,34.016,2.1052082248002,9604487,..
4,Afghanistan,AFG,1964,..,..,50.831,29.867,..,..,..,34.494,2.16119547210755,9814318,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Afghanistan,AFG,2019,97.7,17731939790.9716,36.466,6.791,..,..,496.6025042585,63.565,2.98438913621969,37856121,4.1
60,Afghanistan,AFG,2020,97.7,18585819842.831,36.051,7.113,..,10.4208171386207,510.787063366811,62.575,3.15360920291972,39068979,..
61,Afghanistan,AFG,2021,97.7,13621070038.579,35.842,7.344,..,14.342152797537,356.496214115892,61.982,2.35609779179185,40000412,..
62,Afghanistan,AFG,2022,85.3,..,35.143,6.906,..,18.3800424016375,357.261152798144,62.879,1.43570437048699,40578842,..


218

#Gérer les NaN avec missMDA ...
